In [40]:
import spacy
import pandas as pd
import pandas_ml as pdml
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('hedgewords')
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#from nltk.corpus import hedge_words
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [41]:
nlp = spacy.load('en_core_web_md')

In [42]:
data = pd.read_csv("P2_Training_Dataset.csv") 

post=data['text']
ide=data['thread_id']
label=data['delta']
auth = data['author']

In [43]:
#sampling the dataset to get rid of unimportant datapoints

data = data.dropna(how='any', subset=['author_flair','text'])
data.to_csv(r'C:\Users\Sharvil\feature1\sampled_training_set.csv')

In [44]:
data = pd.read_csv("sampled_training_set.csv") 
data = data.iloc[0:800]
post=data['text']
ide=data['thread_id']
label=data['delta']
auth = data['author']

In [14]:
#dataml = pdml.ModelFrame(data.to_dict(orient='list'))
#sampler = dataml.imbalance.under_sampling.ClusterCentroids()
#sampled = dataml.fit_sample(sampler)

In [45]:
analyser = SentimentIntensityAnalyzer()
def sentiment(s2):
    score = analyser.polarity_scores(s2)
    listscore = list(score.values())
    return listscore

In [46]:
#Feature - Sentiment analysis score
op=0
for i in range(1,ide.size):
    if ide.loc[i]!=ide.loc[op]:
        op=i
        continue
        
    if auth.loc[op]==auth.loc[i]:
        continue
      
    
    s1=str(post.loc[op])
    
    f3=sentiment(s1)
    data.loc[op, 'neg_score'] = f3[0]*100
    data.loc[op, 'neut_score'] = f3[1]*100
    data.loc[op, 'pos_score'] = f3[2]*100
    data.loc[op, 'comp_score'] = f3[3]*100
    
    s2=str(post.loc[i])
    lab=label.loc[i]
    
    
    f3=sentiment(s2)
    data.loc[i, 'neg_score'] = f3[0]*100
    data.loc[i, 'neut_score'] = f3[1]*100
    data.loc[i, 'pos_score'] = f3[2]*100
    data.loc[i, 'comp_score'] = f3[3]*100
    
    
data.to_csv(r'C:\Users\Sharvil\feature1\cleantrainingdata.csv')

In [47]:
data = data.dropna(how='any',subset=['neg_score'])
data.to_csv(r'C:\Users\Sharvil\feature1\cleantrainingdata.csv')

In [48]:
data = pd.read_csv("cleantrainingdata.csv") 
post=data['text']
ide=data['thread_id']
label=data['delta']
auth = data['author']

In [49]:
#Feature - Length
op=0;
for i in range(1,ide.size):
    if ide.loc[i]!=ide.loc[op]:
        op=i
        continue
        
    if auth.loc[op]==auth.loc[i]:
        continue
      
    
    s1=str(post.loc[op])
    data.loc[op, 'length'] = len(s1)
    
    
    s2=str(post.loc[i])
    lab=label.loc[i]
    data.loc[i, 'length'] = len(s2)
    
    
    
    
data.to_csv(r'C:\Users\Sharvil\feature1\cleantrainingdata.csv')

In [50]:
def cosine(s1,s2):
    s1_vec = nlp(s1)
    s2_vec = nlp(s2)
    sim =  s1_vec.similarity(s2_vec)
    return sim*100

In [51]:
#Feature - Cosine Similarity
op=0;
for i in range(1,ide.size):
    if ide.loc[i]!=ide.loc[op]:
        op=i
        continue
        
    if auth.loc[op]==auth.loc[i]:
        continue
      
    
    s1=str(post.loc[op])
    f2=cosine(s1,s1)
    data.loc[op, 'similarity'] = f2
    
    
    s2=str(post.loc[i])
    lab=label.loc[i]
    f2=cosine(s1,s2)
    data.loc[i, 'similarity'] = f2
    
    
    
data.to_csv(r'C:\Users\Sharvil\feature1\cleantrainingdata.csv')

C:\ProgramData\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


In [52]:
#Feature - Count of Hedge words
hdf = pd.read_csv("hedge_list.csv")
hlist=[]
hlista= list(hdf['hedgewords'])
hlist=str(hlista)
#print(type(hlist))

for i in range(1,ide.size):
    count=0
    p = str(post.loc[i])
    wlist = p.split()
    for x in wlist:
        for y in hlist:
            if x==y:
                count=count+1
    data.loc[i, 'count_hedgewords']= count
    

    
data.to_csv(r'C:\Users\Sharvil\feature1\cleantrainingdata.csv')

In [53]:
#undersampling 
minority_num = len(data[data.delta==1])
minority_indices = data[data.delta==1].index
minority_sample = data.loc[minority_indices]

majority_indices = data[data.delta==0].index
random_indices = np.random.choice(majority_indices,minority_num, replace=False)
majority_sample = data.loc[random_indices]

merged_sample = pd.concat([minority_sample,majority_sample],ignore_index=True)
#print(merged_sample.head())
data = merged_sample


In [73]:
xtrain=[]
ytrain = []

l = data['length']
sim = data['similarity']
neg = data['neg_score']
neut = data['neut_score']
pos = data['pos_score']
comp = data['comp_score']
hed = data['count_hedgewords']
post=data['text']
ide=data['thread_id']
label=data['delta']
auth = data['author']

for i in range(1, ide.size):
    v = [l.loc[i],sim.loc[i],neg.loc[i],neut.loc[i],pos.loc[i],comp.loc[i],hed.loc[i]]
    lab = label.loc[i]
    xtrain.append(v)
    ytrain.append(lab)

In [74]:
xtrain=np.array(xtrain).astype(np.int32)
ytrain=np.array(ytrain).astype(np.int32)

rfc = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features='auto')
rfc.fit(xtrain, ytrain)
#print('Random Forest percentage score on 25% validation=',rfc.score(xtrain,ytrain)*100)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [56]:
#TESTING DATASET
datat = pd.read_csv("P2_Testing_Dataset.csv") 
postt=datat['text']
idet=datat['thread_id']
labelt=datat['delta']
autht = datat['author']

In [57]:
#sampling the dataset to get rid of unimportant datapoints

datat = datat.dropna(how='any', subset=['author_flair','text'])
datat.to_csv(r'C:\Users\Sharvil\feature1\sampled_testing_set.csv')

In [58]:
datat = pd.read_csv("sampled_testing_set.csv") 
datat = datat.iloc[0:800]
postt=datat['text']
idet=datat['thread_id']
labelt=datat['delta']
autht = datat['author']

In [59]:
#Feature - Sentiment analysis score
opt=0
for i in range(1,idet.size):
    if idet.loc[i]!=idet.loc[opt]:
        opt=i
        continue
        
    if autht.loc[opt]==autht.loc[i]:
        continue
      
    
    s1t=str(postt.loc[opt])
    
    f3=sentiment(s1t)
    datat.loc[opt, 'neg_score'] = f3[0]*100
    datat.loc[opt, 'neut_score'] = f3[1]*100
    datat.loc[opt, 'pos_score'] = f3[2]*100
    datat.loc[opt, 'comp_score'] = f3[3]*100
    
    s2t=str(postt.loc[i])
    labt=labelt.loc[i]
    
    
    f3=sentiment(s2t)
    datat.loc[i, 'neg_score'] = f3[0]*100
    datat.loc[i, 'neut_score'] = f3[1]*100
    datat.loc[i, 'pos_score'] = f3[2]*100
    datat.loc[i, 'comp_score'] = f3[3]*100
    
    
datat.to_csv(r'C:\Users\Sharvil\feature1\cleantestingdata.csv')

In [60]:
datat = datat.dropna(how='any',subset=['neg_score'])
datat.to_csv(r'C:\Users\Sharvil\feature1\cleantestingdata.csv')

In [61]:
datat = pd.read_csv("cleantestingdata.csv") 
postt=datat['text']
idet=datat['thread_id']
labelt=datat['delta']
autht = datat['author']

In [62]:
#Feature - Length
opt=0
for i in range(1,idet.size):
    if idet.loc[i]!=idet.loc[opt]:
        opt=i
        continue
        
    if autht.loc[opt]==autht.loc[i]:
        continue
      
    
    s1t=str(postt.loc[opt])
    datat.loc[opt, 'length'] = len(s1t)
    
    
    s2t=str(postt.loc[i])
    labt=labelt.loc[i]
    datat.loc[i, 'length'] = len(s2t)
    
    
    
    
datat.to_csv(r'C:\Users\Sharvil\feature1\cleantestingdata.csv')

In [63]:
#Feature - Cosine Similarity
opt=0
for i in range(1,idet.size):
    if idet.loc[i]!=idet.loc[opt]:
        opt=i
        continue
        
    if autht.loc[opt]==autht.loc[i]:
        continue
      
    
    s1t=str(postt.loc[opt])
    f2=cosine(s1t,s1t)
    datat.loc[opt, 'similarity'] = f2
    
    
    s2t=str(postt.loc[i])
    labt=labelt.loc[i]
    f2=cosine(s1t,s2t)
    datat.loc[i, 'similarity'] = f2
    
    
    
datat.to_csv(r'C:\Users\Sharvil\feature1\cleantestingdata.csv')

C:\ProgramData\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\ProgramData\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


In [64]:
#Feature - Count of Hedge words
hdf = pd.read_csv("hedge_list.csv")
hlist=[]
hlista= list(hdf['hedgewords'])
hlist=str(hlista)
#print(type(hlist))

for i in range(1,idet.size):
    count=0
    p = str(postt.loc[i])
    wlist = p.split()
    for x in wlist:
        for y in hlist:
            if x==y:
                count=count+1
    datat.loc[i, 'count_hedgewords']= count
    

    
datat.to_csv(r'C:\Users\Sharvil\feature1\cleantestingdata.csv')

In [69]:
#undersampling 
minority_numt = len(datat[datat.delta==1])
minority_indicest = datat[datat.delta==1].index
minority_samplet = datat.loc[minority_indicest]

majority_indicest = datat[datat.delta==0].index
random_indicest = np.random.choice(majority_indicest,minority_numt, replace=False)
majority_samplet = datat.loc[random_indicest]

merged_samplet = pd.concat([minority_samplet,majority_samplet],ignore_index=True)
#print(merged_sample.head())
datat = merged_samplet
print(datat)

    Unnamed: 0  Unnamed: 0.1 thread_id comment_id reply_to  \
0            0             1    2tijn9    cnzd976   2tijn9   
1           20            51    2tj4pv    cnzy3ou   2tj4pv   
2           42           126    2tjv6i    cnzozux   2tjv6i   
3           70           206    2tlyzs    co15kpv   2tlyzs   
4           71           213    2tlyzs    co0mpip   2tlyzs   
5           82           288    2tlyzs    co0bhvs  co092b2   
6          102           339    2tlyzs    co0lgzu  co0icbg   
7          118           396    2tlyzs    co0nli1  co0fdr2   
8          119           402    2tn8qu    co0jf5s   2tn8qu   
9          217           693    2tpl0x    co1i4k5   2tpl0x   
10         220           705    2tpl0x    co1enmk  co1d15j   
11         240           799    2tquws    co1i3vy   2tquws   
12         345          1077    2tuaj2    co2fk1u   2tuaj2   
13         571          1553    2tuej1    co2ownm  co2nzpw   
14         574          1557    2tuej1    co2pqwp  co2nzpw   
15      

45        2.7      -22.63  1546.0   98.782267             540.0  


In [72]:
xtest=[]
ytest = []

lt = datat['length']
simt = datat['similarity']
negt = datat['neg_score']
neutt = datat['neut_score']
post = datat['pos_score']
compt = datat['comp_score']
hedt = datat['count_hedgewords']
postt=datat['text']
idet=datat['thread_id']
labelt=datat['delta']
autht = datat['author']

for i in range(1, idet.size):
    vt = [lt.loc[i],simt.loc[i],negt.loc[i],neutt.loc[i],post.loc[i],compt.loc[i],hedt.loc[i]]
    labt = labelt.loc[i]
    xtest.append(vt)
    ytest.append(labt)
    
print(xtest)
print(ytest)

[[856.0, 93.48767908214589, 15.2, 76.1, 8.7, -92.74, 120.0], [2986.0, 100.0, 13.8, 72.39999999999998, 13.7, -82.95, 1020.0], [2686.0, 94.0567381848059, 8.200000000000001, 79.5, 12.3, 96.87, 720.0], [1470.0, 86.08103735163836, 3.6, 84.2, 12.2, 93.0, 360.0], [975.0, 95.39942982046624, 8.5, 88.7, 2.8000000000000003, -85.27, 300.0], [1847.0, 94.68562450299133, 2.5, 91.6, 6.0, 81.3, 300.0], [184.0, 89.71737946664128, 10.3, 89.7, 0.0, -47.67, 60.0], [4864.0, 100.0, 11.1, 77.8, 11.1, -79.97, 780.0], [4160.0, 98.15679831948442, 6.9, 80.4, 12.7, 99.18, 1020.0], [2561.0, 98.58509198924814, 19.2, 75.4, 5.4, -99.73, 660.0], [1482.0, 100.0, 15.0, 69.69999999999999, 15.3, -51.13, 120.0], [1730.0, 98.09183791253048, 7.7, 75.8, 16.5, 98.38, 420.0], [798.0, 97.6413481632323, 1.4, 93.8, 4.7, 28.69, 180.0], [2572.0, 98.25891936701022, 4.4, 86.7, 8.9, 96.62, 960.0], [1010.0, 100.0, 2.1, 86.6, 11.2, 92.66, 300.0], [43.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0], [419.0, 94.89571784507831, 4.4, 71.6, 24.0, 96.26, 0.

In [75]:
ypred = rfc.predict(xtest)
print(rfc.score(xtest,ytest)*100)

#Save ypred in csv file


64.44444444444444


In [76]:
#Performance Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


print('\n Confusion Matrix: \n', confusion_matrix(ytest,ypred,labels=[0,1]))
print('\n Accuracy: ', accuracy_score(ytest,ypred)*100)
print('\n Precision, Recall, F1-score: \n',classification_report(ytest,ypred))
s = cohen_kappa_score(ytest,ypred)
print(s)


 Confusion Matrix: 
 [[16  7]
 [ 9 13]]

 Accuracy:  64.44444444444444

 Precision, Recall, F1-score: 
               precision    recall  f1-score   support

           0       0.64      0.70      0.67        23
           1       0.65      0.59      0.62        22

   micro avg       0.64      0.64      0.64        45
   macro avg       0.65      0.64      0.64        45
weighted avg       0.64      0.64      0.64        45

0.28712871287128705
